In [263]:
year = 2023
day = 24

In [264]:
%pip install sympy


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [265]:
from aocd import submit
from aocd.models import Puzzle
from functools import reduce
import numpy as np

np.set_printoptions(edgeitems=30, linewidth=100000,
                    formatter=dict(float=lambda x: "%s" % x))

puzzle = Puzzle(year=year, day=day)
data = puzzle.input_data
left_bound = 200_000_000_000_000
right_bound = 400_000_000_000_000
# data = puzzle.examples[0].input_data
# left_bound = 7
# right_bound = 27

data = data.strip()
data = data.split("\n")
data = [list(map(int, d.replace("@", "").replace(",", "").split())) for d in data]
data[:5]

[[291493672529314, 259618209733833, 379287136024123, -9, 119, -272],
 [308409248682955, 156803514643857, 424989308414284, -78, 236, -255],
 [195379943194796, 213851381371727, 355270583377422, 25, 14, -15],
 [297329579961934, 122004770593749, 344090716183747, -87, 185, -36],
 [295385164557865, 339802914312939, 293784344149228, 64, 10, -211]]

In [266]:
def solve(left, right):
    a, b, c, d = left[3], left[0], right[3], right[0]
    e, f, g, h = left[4], left[1], right[4], right[1]
    try:
        t2 = (a * (h - f) + e * (b - d)) / (c * e - g * a)
        t1 = (t2 * c + d - b) / a
    except ZeroDivisionError:
        # parallel lines
        return None, None, None, None

    xl = t1 * a + b
    xb = t2 * c + d
    yl = t1 * e + f
    yb = t2 * g + h

    return xl, yl, t1, t2

In [267]:
def check(l, r):
    x, y, t1, t2 = solve(l, r)
    if not x:
        return False
    return (left_bound <= x <= right_bound) and (left_bound <= y <= right_bound) and (t1 >= 0) and (t2 >= 0)

In [268]:
from itertools import combinations

sum([check(l, r) for l, r in combinations(data, 2)])

17776

In [269]:
from sympy import solve, Symbol

t1 = Symbol("t1")
t2 = Symbol("t2")
a = Symbol("a")
b = Symbol("b")
c = Symbol("c")
d = Symbol("d")
e = Symbol("e")
f = Symbol("f")
g = Symbol("g")
h = Symbol("h")

solve([t1 * a + b - (t2 * c + d), t1 * e + f - (t2 * g + h)], [t1, t2])

{t1: (-b*g + c*f - c*h + d*g)/(a*g - c*e),
 t2: (a*f - a*h - b*e + d*e)/(a*g - c*e)}

In [270]:
from sympy import solve, Symbol

t1 = Symbol("t1")
t2 = Symbol("t2")
a1 = Symbol("a1")
b1 = Symbol("b1")
c1 = Symbol("c1")
d1 = Symbol("d1")
a2 = Symbol("a2")
b2 = Symbol("b2")
c2 = Symbol("c2")
d2 = Symbol("d2")

alpha = Symbol("alpha")
beta = Symbol("beta")
upsilon = Symbol("upsilon")
delta = Symbol("delta")

sol = solve([a1 * t1 + c1 - (alpha * t1 + upsilon),
             b1 * t1 + d1 - (beta * t1 + delta),
             a2 * t2 + c2 - (alpha * t2 + upsilon),
             b2 * t2 + d2 - (beta * t2 + delta)],
            [t1,
             t2,
             delta,
             upsilon])
sol

{delta: (a1*b2*d1 - a1*beta*d1 - a2*b1*d2 + a2*beta*d2 + alpha*b1*d2 - alpha*b2*d1 + alpha*beta*d1 - alpha*beta*d2 - b1*b2*c1 + b1*b2*c2 + b1*beta*c1 - b1*beta*c2 + b2*beta*c1 - b2*beta*c2 - beta**2*c1 + beta**2*c2)/(a1*b2 - a1*beta - a2*b1 + a2*beta + alpha*b1 - alpha*b2),
 t1: (a2*d1 - a2*d2 - alpha*d1 + alpha*d2 - b2*c1 + b2*c2 + beta*c1 - beta*c2)/(a1*b2 - a1*beta - a2*b1 + a2*beta + alpha*b1 - alpha*b2),
 t2: (a1*d1 - a1*d2 - alpha*d1 + alpha*d2 - b1*c1 + b1*c2 + beta*c1 - beta*c2)/(a1*b2 - a1*beta - a2*b1 + a2*beta + alpha*b1 - alpha*b2),
 upsilon: (a1*a2*d1 - a1*a2*d2 - a1*alpha*d1 + a1*alpha*d2 + a1*b2*c2 - a1*beta*c2 - a2*alpha*d1 + a2*alpha*d2 - a2*b1*c1 + a2*beta*c1 + alpha**2*d1 - alpha**2*d2 + alpha*b1*c1 - alpha*b2*c2 - alpha*beta*c1 + alpha*beta*c2)/(a1*b2 - a1*beta - a2*b1 + a2*beta + alpha*b1 - alpha*b2)}

In [271]:
compute_delta = lambdify([a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta], sol[delta])
compute_upsilon = lambdify([a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta], sol[upsilon])
compute_t1 = lambdify([a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta], sol[t1])
compute_t2 = lambdify([a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta], sol[t2])

In [272]:
compute_delta = lambda a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta: (
                                                                            a1 * b2 * d1 - a1 * beta * d1 - a2 * b1 * d2 + a2 * beta * d2 + alpha * b1 * d2 - alpha * b2 * d1 + alpha * beta * d1 - alpha * beta * d2 - b1 * b2 * c1 + b1 * b2 * c2 + b1 * beta * c1 - b1 * beta * c2 + b2 * beta * c1 - b2 * beta * c2 - beta ** 2 * c1 + beta ** 2 * c2) / (
                                                                            a1 * b2 - a1 * beta - a2 * b1 + a2 * beta + alpha * b1 - alpha * b2)

compute_t1 = lambda a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta: (
                                                                         a2 * d1 - a2 * d2 - alpha * d1 + alpha * d2 - b2 * c1 + b2 * c2 + beta * c1 - beta * c2) / (
                                                                         a1 * b2 - a1 * beta - a2 * b1 + a2 * beta + alpha * b1 - alpha * b2)

compute_t2 = lambda a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta: (
                                                                         a1 * d1 - a1 * d2 - alpha * d1 + alpha * d2 - b1 * c1 + b1 * c2 + beta * c1 - beta * c2) / (
                                                                         a1 * b2 - a1 * beta - a2 * b1 + a2 * beta + alpha * b1 - alpha * b2)

compute_upsilon = lambda a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta: (
                                                                              a1 * a2 * d1 - a1 * a2 * d2 - a1 * alpha * d1 + a1 * alpha * d2 + a1 * b2 * c2 - a1 * beta * c2 - a2 * alpha * d1 + a2 * alpha * d2 - a2 * b1 * c1 + a2 * beta * c1 + alpha ** 2 * d1 - alpha ** 2 * d2 + alpha * b1 * c1 - alpha * b2 * c2 - alpha * beta * c1 + alpha * beta * c2) / (
                                                                              a1 * b2 - a1 * beta - a2 * b1 + a2 * beta + alpha * b1 - alpha * b2)

In [273]:
SEARCH_RANGE = 300
for alpha in range(-SEARCH_RANGE, SEARCH_RANGE):
    for beta in range(-SEARCH_RANGE, SEARCH_RANGE):

        deltas = []
        upsilons = []

        N = 3

        for i in range(N):

            l, r = data[i], data[i + 1]
            a1, b1, c1, d1, e1, f1 = l[3], l[4], l[0], l[1], l[5], l[2]
            a2, b2, c2, d2, e2, f2 = r[3], r[4], r[0], r[1], r[5], r[2]

            try:
                delta = compute_delta(a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta)
                upsilon = compute_upsilon(a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta)
            except ZeroDivisionError:
                continue

            deltas.append(delta)
            upsilons.append(upsilon)

        if np.ptp(deltas) < 0.001 and np.ptp(upsilons) < 0.001 and len(deltas) == N:
            t1 = compute_t1(a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta)
            t2 = compute_t2(a1, a2, b1, b2, c1, c2, d1, d2, alpha, beta)
            print(f"alpha: {alpha}, beta: {beta} upsilon: {int(upsilon)} delta: {int(delta)} t1: {t1} t2: {t2}")
            sol_variables = {
                "alpha": alpha,
                "beta": beta,
                "upsilon": upsilon,
                "delta": delta,
                "t1": t1,
                "t2": t2,
                "a1": a1,
                "b1": b1,
                "c1": c1,
                "d1": d1,
                "e1": e1,
                "f1": f1,
                "a2": a2,
                "b2": b2,
                "c2": c2,
                "d2": d2,
                "e2": e2,
                "f2": f2,

            }

        # print(alpha, beta, deltas, upsilons)

alpha: -193, beta: -230 upsilon: 370994826025810 delta: 410411158485339 t1: 805572857023.0 t2: 694955151546.0


In [274]:
sol_variables

{'alpha': -193,
 'beta': -230,
 'upsilon': 370994826025810.0,
 'delta': 410411158485339.0,
 't1': 805572857023.0,
 't2': 694955151546.0,
 'a1': 25,
 'b1': 14,
 'c1': 195379943194796,
 'd1': 213851381371727,
 'e1': -15,
 'f1': 355270583377422,
 'a2': -87,
 'b2': 185,
 'c2': 297329579961934,
 'd2': 122004770593749,
 'e2': -36,
 'f2': 344090716183747}

In [275]:
from sympy import solve, Symbol

t1 = Symbol("t1")
t2 = Symbol("t2")
a1 = Symbol("a1")
b1 = Symbol("b1")
c1 = Symbol("c1")
d1 = Symbol("d1")
e1 = Symbol("e1")
f1 = Symbol("f1")
a2 = Symbol("a2")
b2 = Symbol("b2")
c2 = Symbol("c2")
d2 = Symbol("d2")
e2 = Symbol("e2")
f2 = Symbol("f2")

alpha = Symbol("alpha")
beta = Symbol("beta")
upsilon = Symbol("upsilon")
delta = Symbol("delta")
gamma = Symbol("gamma")
epsilon = Symbol("epsilon")

sol = solve([a1 * t1 + c1 - (alpha * t1 + upsilon),
             b1 * t1 + d1 - (beta * t1 + delta),
             e1 * t1 + f1 - (gamma * t1 + epsilon),
             a2 * t2 + c2 - (alpha * t2 + upsilon),
             b2 * t2 + d2 - (beta * t2 + delta),
             e2 * t2 + f2 - (gamma * t2 + epsilon)
             ],
            [gamma, epsilon])
sol

{epsilon: (-e1*t1*t2 + e2*t1*t2 - f1*t2 + f2*t1)/(t1 - t2),
 gamma: (e1*t1 - e2*t2 + f1 - f2)/(t1 - t2)}

In [276]:
compute_epsilon = lambda e1, f1, e2, f2, t1, t2: (-e1 * t1 * t2 + e2 * t1 * t2 - f1 * t2 + f2 * t1) / (t1 - t2)
compute_gamma = lambda e1, f1, e2, f2, t1, t2: (e1 * t1 - e2 * t2 + f1 - f2) / (t1 - t2)

In [277]:
sol_variables["epsilon"] = compute_epsilon(sol_variables["e1"], sol_variables["f1"], sol_variables["e2"], sol_variables["f2"], sol_variables["t1"], sol_variables["t2"])

In [278]:
sol_variables["gamma"] = compute_gamma(sol_variables["e1"], sol_variables["f1"], sol_variables["e2"], sol_variables["f2"], sol_variables["t1"], sol_variables["t2"])

In [279]:
print(f"alpha: {sol_variables['alpha']}, beta: {sol_variables['beta']}, upsilon: {int(sol_variables['upsilon'])}, delta: {int(sol_variables['delta'])}, gamma: {int(sol_variables['gamma'])}, epsilon: {int(sol_variables['epsilon'])}, t1: {sol_variables['t1']}, t2: {sol_variables['t2']}")

alpha: -193, beta: -230, upsilon: 370994826025810, delta: 410411158485339, gamma: 218, epsilon: 167572107691063, t1: 805572857023.0, t2: 694955151546.0


In [280]:
int(sol_variables["upsilon"] + sol_variables["delta"] + sol_variables["epsilon"])

948978092202212

In [281]:
from sympy.utilities.lambdify import lambdify

In [282]:
r = lambdify([e1, f1, e2, f2, t1, t2], sol[epsilon])

In [283]:
r(sol_variables["e1"], sol_variables["f1"], sol_variables["e2"], sol_variables["f2"], sol_variables["t1"], sol_variables["t2"])

167572107691063.22

In [284]:
compute_epsilon(sol_variables["e1"], sol_variables["f1"], sol_variables["e2"], sol_variables["f2"], sol_variables["t1"], sol_variables["t2"])

167572107691063.22